In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter


In [14]:

data=pd.read_csv('spam_clean.csv',encoding='latin-1')
data.head()

,type,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


# cleaning messages

In [15]:
import nltk 
import re
nltk.download('punkt_tab')
nltk.download('stopwords')
from nltk import word_tokenize,sent_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\apattan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\apattan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
def clean_message(sentence):
  words=nltk.word_tokenize(sentence)
  cleaned_msg=""
  for word in words:
    word=word.lower()
    word=re.sub(r'[^a-z]','',word)
    if word!="" and word not in stopwords.words('english'):
      cleaned_msg=cleaned_msg+" "+word
  return cleaned_msg.strip()
  
    

In [17]:
data['cleaned_msg']=data['message'].apply(clean_message)
data.head()

,type,message,cleaned_msg
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry wkly comp win fa cup final tkts st ...
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah nt think goes usf lives around though


In [18]:
data.shape

(5572, 3)

# data preparation for naive bayes

In [19]:
class NaiveBayes:
    def __init__(self):
        
        self.alpha=1

    
    def fit(self,X,y,common_words=10):
        self.X=X
        self.y=y

        self.prior_prob=self.y.value_counts(normalize=True)
        ham=""
        spam=""
        ham=ham.join(self.X[self.y=='ham'])
        ham_words=ham.split()
        spam=spam.join(self.X[self.y=='spam'])
        spam_words=spam.split()

        ham_word_count=np.array(Counter(ham_words).most_common(common_words))[:,0]
        spam_word_count=np.array(Counter(spam_words).most_common(common_words))[:,0]

        self.features=ham_word_count
        self.features=np.append(self.features,spam_word_count)
        self.features=np.unique(self.features)
        

        count_matrix=pd.DataFrame(np.zeros((self.X.shape[0],self.features.shape[0])))
        count_matrix.columns=self.features

        self.word_frequency_matrix= pd.concat([self.X,count_matrix],axis=1)
        self.word_count()
        self.word_frequency_matrix.drop(columns='cleaned_msg',inplace=True)

        self.word_prob=pd.DataFrame(np.sum(self.word_frequency_matrix)/np.sum(self.word_frequency_matrix).sum(),columns=["total"])
        self.word_prob["ham"]=np.sum(self.word_frequency_matrix[self.y=="ham"])/np.sum(self.word_frequency_matrix[self.y=="ham"]).sum()
        self.word_prob["spam"]=np.sum(self.word_frequency_matrix[self.y=="spam"])/np.sum(self.word_frequency_matrix[self.y=="spam"]).sum()

        self.word_prob.loc[self.word_prob['ham']==0,'ham']=self.alpha
        self.word_prob.loc[self.word_prob['spam']==0,'spam']=self.alpha
    
    def predict(self,query):
        # Assuming query is a clean message
        words=self.give_me_words_from_features(query)
        spam_prob=(np.prod(self.word_prob.loc[words,"spam"])*self.prior_prob["spam"])/(np.prod(self.word_prob.loc[words,"total"]))
        ham_prob=(np.prod(self.word_prob.loc[words,"ham"])*self.prior_prob["ham"])/(np.prod(self.word_prob.loc[words,"total"]))

        print(np.prod(self.word_prob.loc[words,"spam"]))
        print(np.prod(self.word_prob.loc[words,"ham"]))
        
        if(spam_prob>ham_prob):
            return "spam"
        else:
            return "ham"
        

    def word_count(self):
        for idx,row in enumerate(self.word_frequency_matrix['cleaned_msg']):
            word_count=Counter(self.give_me_words_from_features(row))
            self.word_frequency_matrix.loc[idx,word_count.keys()]=word_count.values()

    def give_me_words_from_features(self,sentence):
        return [word for word in sentence.split() if word in self.features]

In [20]:
NB=NaiveBayes()
NB.fit(data['cleaned_msg'],data['type'])
print(NB.predict("Hi how are u?"))

1.0
1.0
ham


c:\Users\apattan\OneDrive - Mercedes-Benz (corpdir.onmicrosoft.com)\My documents\coding\data\.venv\lib\site-packages\numpy\_core\fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
c:\Users\apattan\OneDrive - Mercedes-Benz (corpdir.onmicrosoft.com)\My documents\coding\data\.venv\lib\site-packages\numpy\_core\fromnumeric.py:84: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
